In [2]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori,association_rules

In [3]:
data=pd.read_excel('Online_RetailApriori.xlsx')    #pip install apyori---->pip install mlxtend---->pip install openpyxl

In [4]:
print(data)

       InvoiceNo StockCode                          Description  Quantity  \
0         536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1         536365     71053                  WHITE METAL LANTERN         6   
2         536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3         536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4         536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
...          ...       ...                                  ...       ...   
541904    581587     22613          PACK OF 20 SPACEBOY NAPKINS        12   
541905    581587     22899         CHILDREN'S APRON DOLLY GIRL          6   
541906    581587     23254        CHILDRENS CUTLERY DOLLY GIRL          4   
541907    581587     23255      CHILDRENS CUTLERY CIRCUS PARADE         4   
541908    581587     22138        BAKING SET 9 PIECE RETROSPOT          3   

               InvoiceDate  UnitPrice  CustomerID         Country  
0      

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [6]:
# Exploring the columns of the data
data.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [7]:
# Exploring the different regions of transactions
data.Country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [8]:
#Cleaning the data

#striping the extra spaces in the description
data['Description'] = data['Description'].str.strip()

In [9]:
## Dropping the rows without any invoice number
data.dropna(axis = 0,subset = ['InvoiceNo'],inplace = True)
data['InvoiceNo'] = data['InvoiceNo'].astype('str')

In [10]:
# Dropping all transactions which were done on credit
data = data[~data['InvoiceNo'].str.contains('C')]

In [20]:
#Splitting the data according to the region of transaction

# Transactions done in France
basket_France = (data[data['Country'] =="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [21]:
print(basket_France)

Description  10 COLOUR SPACEBOY PEN  12 COLOURED PARTY BALLOONS  \
InvoiceNo                                                         
536370                          0.0                         0.0   
536852                          0.0                         0.0   
536974                          0.0                         0.0   
537065                          0.0                         0.0   
537463                          0.0                         0.0   
...                             ...                         ...   
580986                          0.0                         0.0   
581001                          0.0                         0.0   
581171                          0.0                         0.0   
581279                          0.0                         0.0   
581587                          0.0                         0.0   

Description  12 EGG HOUSE PAINTED WOOD  12 MESSAGE CARDS WITH ENVELOPES  \
InvoiceNo                                            

In [22]:
# Transactions done in the United Kingdom
basket_UK = (data[data['Country'] =="United Kingdom"]            #removing unstack()
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [23]:
print(basket_UK)

                                   Description  Quantity
InvoiceNo                                               
536365          CREAM CUPID HEARTS COAT HANGER         8
536365       GLASS STAR FROSTED T-LIGHT HOLDER         6
536365     KNITTED UNION FLAG HOT WATER BOTTLE         6
536365          RED WOOLLY HOTTIE WHITE HEART.         6
536365            SET 7 BABUSHKA NESTING BOXES         2
...                                        ...       ...
581586           RED RETROSPOT ROUND CAKE TINS        24
581586        SET OF 3 HANGING OWLS OLLIE BEAK        24
A563185                        Adjust bad debt         1
A563186                        Adjust bad debt         1
A563187                        Adjust bad debt         1

[475560 rows x 2 columns]


In [25]:
# Defining the hot encoding function to make the data suitable for the concerned libraries
def hot_encode(x):
    if(x<= 0):
        return 0
    if(x>= 1):
        return 1
    
# Encoding the datasets
basket_encoded = basket_France.applymap(hot_encode)
basket_France = basket_encoded

In [26]:
# Building the model
frq_items = apriori(basket_France, min_support = 0.05, use_colnames = True)
  
# Collecting the inferred rules in a dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
print(rules.head())

                                           antecedents  \
45                        (JUMBO BAG WOODLAND ANIMALS)   
258  (RED TOADSTOOL LED NIGHT LIGHT, PLASTERS IN TI...   
270  (RED TOADSTOOL LED NIGHT LIGHT, PLASTERS IN TI...   
302  (SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...   
300  (SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...   

                         consequents  antecedent support  consequent support  \
45                         (POSTAGE)            0.076531            0.765306   
258                        (POSTAGE)            0.051020            0.765306   
270                        (POSTAGE)            0.053571            0.765306   
302  (SET/6 RED SPOTTY PAPER PLATES)            0.102041            0.127551   
300    (SET/6 RED SPOTTY PAPER CUPS)            0.102041            0.137755   

      support  confidence      lift  leverage  conviction  zhangs_metric  
45   0.076531       1.000  1.306667  0.017961         inf       0.254144  
258  0.051020       

C:\Users\cclab34\AppData\Local\Programs\Python\Python310\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
